In [ ]:
import numpy as np
import pandas as pd

import nltk
nltk.download('popular')

import spacy
from nltk.stem import SnowballStemmer
stem_en = SnowballStemmer("english")

import re

stopwordsenglish = nltk.corpus.stopwords.words("english")
nlp = spacy.load('en_core_web_sm')

import requests


########## test

from wordcloud import WordCloud
import matplotlib.pyplot as plt
import seaborn as sns

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import accuracy_score,precision_score,recall_score,confusion_matrix,roc_curve,classification_report
from scikitplot.metrics import plot_confusion_matrix


#######

import text2emotion as te

In [ ]:
df_august = pd.read_csv("https://raw.githubusercontent.com/CathieLC/Happy_four_friends/main/Dataset/ManoMano%20-%20August%202021.csv?token=GHSAT0AAAAAABMXSZJMU35CES7HVZWAJODMYPRFTPA")
df_september = pd.read_csv("https://raw.githubusercontent.com/CathieLC/Happy_four_friends/main/Dataset/ManoMano%20-%20September%202021.csv?token=GHSAT0AAAAAABMXSZJMUP5FCAR47B6L2AEQYPRFU5Q")
df_october = pd.read_csv("https://raw.githubusercontent.com/CathieLC/Happy_four_friends/main/Dataset/ManoMano%20-%20October%202021.csv?token=GHSAT0AAAAAABMXSZJMRDZONS4ZZPZ4U72UYPRFVFQ")
df_november = pd.read_csv("https://raw.githubusercontent.com/CathieLC/Happy_four_friends/main/Dataset/ManoMano%20-%20November%202021.csv?token=GHSAT0AAAAAABMXSZJM57XIDQDMXJ3CA5IKYPRFVMA")


In [ ]:
# Concatenation of all dataset
frames = [df_august, df_september, df_october, df_november]
df_total = pd.concat(frames)
# Display all columns when exploring datasets
pd.set_option('display.max_columns', None)

In [ ]:
print (df_august.shape)
print (df_september.shape)
print (df_october.shape)
print (df_november.shape)

In [ ]:
df_total["first_order"].value_counts(normalize=True)

In [ ]:
df_total["csat_presales"].value_counts(normalize=True)

In [ ]:
df_total["has_presales_contact"].value_counts(normalize=True)

In [ ]:
df_total["has_manodvisor_contact"].value_counts(normalize=True)

In [ ]:
df_total.head()

In [ ]:
df_total.isna().sum()

In [ ]:
df_comments = df_total.dropna(subset=["comment"])
df_comments

In [ ]:
def func_clean(sentence):
  sentence = re.sub(r'[^\w\s]','',str(sentence))
  sentence = nltk.word_tokenize(sentence.lower())
  newsentence = ""
  tokens_clean = []
  for words in sentence:
    if words not in stopwordsenglish:
      tokens_clean.append(words)
  tokens_clean2 = ' '.join([str(item) for item in tokens_clean])
  sent_tokens = nlp(tokens_clean2)
  for token in sent_tokens:
    newsentence = newsentence + " " + (token.lemma_)
  return newsentence

In [ ]:
df_comments2 = df_comments
df_comments2["comment"] = df_comments2["comment"].apply(func_clean)
df_comments2

In [ ]:
my_text = ""

for i in df_comments2["comment"]:
    my_text+=i
 

In [ ]:
nlp = spacy.load('en_core_web_sm')

sent_tokens = nlp(my_text)

lemma_lst = []

for token in sent_tokens:
  lemma_lst.append(token.lemma_)

lemma_freq = nltk.FreqDist(lemma_lst)

lemma_freq

In [ ]:
wordcloud = WordCloud(width=480, height=480, max_font_size=200, min_font_size=10)


wordcloud.generate_from_frequencies(lemma_freq)

plt.figure(figsize=(12, 10))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

In [ ]:
df_sentiment = df_comments[["id", "comment", "score"]]
df_sentiment

In [ ]:
##df_sentiment.to_csv(r'C:\Users\Berenger\Desktop\team_happy_four_friends\df_sentiment.csv', index = False, header = True)

In [ ]:
print (df_sentiment.iloc[3,1])
te.get_emotion(df_sentiment.iloc[3,1])

In [ ]:
df_sentiment["sentiment"] = df_sentiment["comment"].apply(te.get_emotion)
df_sentiment

In [ ]:
df_sentiment2 = df_sentiment.copy()

In [ ]:
df_sentiment2

In [ ]:
df_sentiment2.reset_index(inplace=True)



In [ ]:
df_sentiment2[["happy", "angry", "surprise", "sad", "fear"]] = 0

In [ ]:
df_sentiment2

In [ ]:
for i in range(len(df_sentiment2["sentiment"])):
    column_number = 5
    for z in df_sentiment2["sentiment"][i]:
        df_sentiment2.iloc[i, column_number] = df_sentiment2["sentiment"][i][z]
        column_number+=1


In [ ]:
df_sentiment2.set_index("index", inplace=True)

In [ ]:
df_comments2["happy"] = df_sentiment2["happy"]
df_comments2["angry"] = df_sentiment2["angry"]
df_comments2["surprise"] = df_sentiment2["surprise"]
df_comments2["sad"] = df_sentiment2["sad"]
df_comments2["fear"] = df_sentiment2["fear"]

In [ ]:
df_comments2.head()

In [ ]:
df_comments3 = df_comments2[df_comments2["happy"] + df_comments2["angry"] + df_comments2["surprise"] + df_comments2["sad"] + df_comments2["fear"] != 0]

In [ ]:
df_comments3.head()

In [ ]:
df_comments3['bv_transaction'] = df_comments3['bv_transaction'].str.replace(',','.')

In [ ]:
df_comments3.head()

In [ ]:
df_comments3["bv_transaction"] = pd.to_numeric(df_comments3["bv_transaction"], downcast="float")
df_comments3["happy"] = pd.to_numeric(df_comments3["happy"], downcast="float")
df_comments3["angry"] = pd.to_numeric(df_comments3["angry"], downcast="float")
df_comments3["surprise"] = pd.to_numeric(df_comments3["surprise"], downcast="float")
df_comments3["sad"] = pd.to_numeric(df_comments3["sad"], downcast="float")
df_comments3["fear"] = pd.to_numeric(df_comments3["fear"], downcast="float")

In [ ]:
df_comments3.head()
df_comments3 = df_comments3[["score", "happy", "angry", "surprise", "sad", "fear"]]

In [ ]:
def plotly_heatmap(df):
    import plotly.express as px
    import plotly.graph_objects as go
    corr = df.corr()
    fig = go.Figure()
    fig.add_trace(go.Heatmap(
        z = corr,
        x = corr.columns.values,
        y = corr.columns.values,
        colorscale = px.colors.diverging.RdBu,
        zmid=0
    ))

    fig.update_layout(width=800, height=700)
    fig.show()

plotly_heatmap(df_comments3)

In [ ]:
df_comments3.corr()